In [1]:
import sys
import itertools
import os
import numpy as np
import torch as tr
import pandas as pd

from CSWSEM import *
from matplotlib import pyplot as plt

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
## input params
learn_rate = 0.05
alfa = 0.0001
lmda = 100000
stsize = 35

In [3]:
gs_name = 'gs-acc'

In [4]:
num_seeds = 2
condL = ['blocked','interleaved']

exp_kwargs={
    'n_train':160,
    'n_test':40
}

sem_kwargs={
    'nosplit':False,
    'stsize':stsize,
    'lmda':lmda,
    'alfa':alfa,
    'learn_rate':learn_rate,
}

In [5]:
dataL = []
exphat = -np.ones([num_seeds,len(condL),200,5,10]) # (trials,tsteps,obsdim)

In [6]:
for seed in np.arange(num_seeds):
  for cidx,cond in enumerate(condL):
    print(seed,cond)
    np.random.seed(seed)
    tr.manual_seed(seed)
    # setup
    sem_kwargs['seed'] = seed
    exp_kwargs['condition'] = cond
    task = CSWTask(seed)
    sem = SEM(**sem_kwargs)
    # run
    exp,curr = task.generate_experiment(**exp_kwargs)
    sem_data = sem.forward_exp(exp,curr)
    ### eval
    exphat[seed,cidx] = np.array([tdata['event_hat'] for tdata in sem_data])
    # record data
    sem.data.record_exp('condition',exp_kwargs['condition'])
    dataL.append(pd.DataFrame(sem_data))

0 blocked
0 interleaved
1 blocked
1 interleaved


In [14]:
save_fpath = 'gsdata/%s/'%(gs_name)
batch_tag = 'learnrate_%.3f__alfa_%f__lmbda_%f'%(learn_rate,alfa,lmda)
np.save(save_fpath+'exphat__'+batch_tag,exphat)
pd.concat(dataL).to_csv(save_fpath+'semdata__'+batch_tag+'.csv')

done
